# JETANK Gamepad
  
If you have a gamepad with a USB interface, you can use this document to control JETANK.

Unlike the teleoperation example provided by Jetbot, you need to connect the USB receiver of the gamepad to the USB port of Jetson Nano instead of connecting to the computer.

After connecting the USB receiver to Jetson Nano, turn on the switch on the gamepad, the indicator light on the gamepad turns on, and then run the following code.
  

### Importing the Camera class
  
Run the following code to open the camera widget. When the screen opens, you can right-click on the screen, select `Create new view for output`, and then drag and drop the screen to a suitable place next to it.

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=300, height=300)

image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

  
### Basic control
  
Since the inverse kinematics function needs to be called continuously to guarantee the end of the robotic arm move in a straight line, it is necessary to import the threading library for multi-threaded control of the robotic arm.

In [2]:
import threading
from time import sleep
from jetbot import Robot
from SCSCtrl import TTLServo

robot = Robot()

Succeeded to open the port
Succeeded to change the baudrate


  
Define some variables and parameters for basic control.

In [3]:
moveStartTor = 0.15    # The motor's initial rotation threshold. 
                       # The geared motor cannot rotate in the low PWM interval. 
                       # Long-term stall will reduce the motor's life.

fb_input = 0           # Store the speed parameter for the forward and backward direction.
lr_input = 0           # Store the parameters for steering.

servoCtrlTime = 0.001  # Delay after the TTL servo communication to avoid errors.
speedInput = 300       # Camera tilt and pan rotation speed.

armXStatus = 0         # The movement state of the X axis of the robotic arm (far and near direction).
armYStatus = 0         # The movement state of the Y axis of the robotic arm (vertical direction).
cameStatus = 0         # The movement state of the Camera.

goalX = 130            # Store the X-axis coordinate of the gripper of the robotic arm.
goalY = 50             # Store the Y-axis coordinate of the gripper of the robotic arm.
goalC = 0              # Store the position of camera.

xMax = 210             # Set the maximum X axis of the robotic arm.
xMin = 140             # Set the minimum X axis of the robotic arm.

yMax = 120             # Set the maximum Y axis of the robotic arm.
yMix = -120            # Set the minimum Y axis of the robotic arm.

cMax = 25              # Set the maximum position of the camera.
cMin = -45             # Set the minimum position of the camera.
cDan = 0               # Set the dangerous line of the camera.

movingTime = 0.005     # Set the running time between every two adjacent positions of the inverse kinematics function of the robotic arm.
movingSpeed = 1        # Set the movement speed of the robotic arm.

grabStatus = 0         # The movement state of the gripper.

cameraPosCheckMode = 1 # The switch of the camera position checking function, 1 - on, 0 - off.
                       # onec it is on, it can avoid the interference between the robotic arm and the camera.
cameraPosDanger = 0    # Whether the camera is in the danger zone.
cameraMoveSafe = 0     # Whether the camera has moved to the safe zone.

  
After running the code below, the robotic arm automatically moves to the position(goalX, goalY), and the remaining servos slowly rotate to the initial position.

In [4]:
TTLServo.xyInputSmooth(goalX, goalY, 3)
TTLServo.servoAngleCtrl(1, 0, 1, 100)
TTLServo.servoAngleCtrl(4, 0, 1, 100)
TTLServo.servoAngleCtrl(5, 0, 1, 100)
sleep(3.5)
print('ready!')

ready!


  
Define some basic exercise fuctions.

In [5]:
# The robotic arm stops the current action.
def armStop():
    global armXStatus, armYStatus
    if armXStatus or armYStatus:
        armXStatus = 0
        armYStatus = 0
        TTLServo.servoStop(2)
        sleep(servoCtrlTime)
        TTLServo.servoStop(3)
        sleep(servoCtrlTime)
        print([goalX, goalY])


# Camera position checking and protection function.
def armCameraPosCheck():
    global cameraPosDanger, cameraMoveSafe, goalC
    if goalY < -50 and cameraPosCheckMode:
        cameraPosDanger = 1
        if cameraMoveSafe and goalC < cDan:
            goalC = cDan
            TTLServo.servoAngleCtrl(5, cDan, 1, speedInput)
            sleep(servoCtrlTime)
            cameraMoveSafe = 0
    else:
        cameraPosDanger = 0


# Control the gripping and loosening of the gripper.
def grabCtrlCommand(commandType, valueInput):
    global grabStatus
    if commandType == 'tl':
        commandLoose = 0
        if valueInput:
            commandGrab = 1
        else:
            commandGrab = 0
    elif commandType == 'tl2':
        commandGrab = 0
        if valueInput:
            commandLoose = 1
        else:
            commandLoose = 0
    else:
        return

    if commandGrab and not grabStatus:
        TTLServo.servoAngleCtrl(4, -90, 1, 150)
        grabStatus = 1
        sleep(servoCtrlTime)
    elif commandLoose and not grabStatus:
        TTLServo.servoAngleCtrl(4, 0, 1, 150)
        grabStatus = 1
        sleep(servoCtrlTime)

    if not commandGrab and not commandLoose and grabStatus:
        TTLServo.servoStop(4)
        grabStatus = 0
        sleep(servoCtrlTime)


# Robotic arm control function, for command input.
def armCtrlCommand(commandType, valueInput):
    global xMax, xMin, yMax, yMix, goalX, goalY, armXStatus, armYStatus
    commandXin, commandXde, commandYin, commandYde = 0, 0, 0, 0
    if commandType == 'y':
        if valueInput:
            ctrlArm.moveXin = 1
            ctrlArm.resume()
            print('run')
        else:
            ctrlArm.moveXin = 0
            ctrlArm.pause()
    elif commandType == 'a':
        if valueInput:
            ctrlArm.moveXde = 1
            ctrlArm.resume()
        else:
            ctrlArm.moveXde = 0
            ctrlArm.resume()
    elif commandType == 'tr':
        if valueInput:
            ctrlArm.moveYin = 1
            ctrlArm.resume()
        else:
            ctrlArm.moveYin = 0
            ctrlArm.resume()
    elif commandType == 'tr2':
        if valueInput:
            ctrlArm.moveYde = 1
            ctrlArm.resume()
        else:
            ctrlArm.moveYde = 0
            ctrlArm.resume()
    else:
        return


# Robotic arm control function, for action execution (in another single thread).
def armCtrl(commandXin, commandXde, commandYin, commandYde):
    global xMax, xMin, yMax, yMix, goalX, goalY, armXStatus, armYStatus, cameraMoveSafe
    armCameraPosCheck()
    if commandXin:
        armXStatus = 1
        goalX += movingSpeed
        if goalX > xMax:
            goalX = xMax
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    elif commandXde:
        armXStatus = 1
        goalX -= movingSpeed
        if goalX < xMin:
            goalX = xMin
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    if commandYin:
        armYStatus = 1
        goalY += movingSpeed
        if goalY > yMax:
            goalY = yMax
        if goalY > -50:
            cameraMoveSafe = 1
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    elif commandYde:
        armYStatus = 1
        goalY -= movingSpeed
        if goalY < yMix:
            goalY = yMix
        if goalY > -50:
            cameraMoveSafe = 1
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        sleep(servoCtrlTime)
        return

    if not commandXin and not commandXde and not commandYin and not commandYde:
        armStop()


# Camera tilting motion stopped.
def tiltStop():
    global cameStatus
    cameStatus = 0
    TTLServo.servoStop(5)
    sleep(servoCtrlTime)


# Camera (robotic arm) stops moving in the pan direction.
def panStop():
    TTLServo.servoStop(1)
    sleep(servoCtrlTime)


# Camera tilt motion control. (in another single thread)
def cameraTiltCtrl(commandCin, commandCde):
    global goalC, cameStatus
    if commandCde:
        cameStatus = 1
        if not cameraPosDanger:
            goalC -= movingSpeed
            if goalC < cMin:
                goalC = cMin
            TTLServo.servoAngleCtrl(5, goalC, 1, speedInput)
            sleep(servoCtrlTime)
        else:
            goalC -= movingSpeed
            if goalC < cDan:
                goalC = cDan
            TTLServo.servoAngleCtrl(5, goalC, 1, speedInput)
            sleep(servoCtrlTime)
        return
    if commandCin:
        cameStatus = 1
        goalC += movingSpeed
        if goalC > cMax:
            goalC = cMax
        TTLServo.servoAngleCtrl(5, goalC, 1, speedInput)
        sleep(servoCtrlTime)
        return
    if not commandCde and not commandCin:
        tiltStop()
            
    
# Camera (robotic arm) pan rotation control.
def cameraPanCtrlCommand(commandType, valueInput, buttonLoop):
    commandLeft, commandRight = 0, 0
    if commandType == 'lr' and buttonLoop:
        if valueInput == -1:
            commandLeft = 1
        elif valueInput == 1:
            commandRight = 1
    elif commandType == 'x' and not buttonLoop:
        if valueInput:
            commandLeft = 1
    elif commandType == 'b' and not buttonLoop:
        if valueInput:
            commandRight = 1
    else:
        return

    if commandLeft:
        TTLServo.servoAngleCtrl(1, -80, 1, speedInput)
        sleep(servoCtrlTime)
    elif commandRight:
        TTLServo.servoAngleCtrl(1, 80, 1, speedInput)
        sleep(servoCtrlTime)
    elif not commandLeft and not commandRight:
        panStop()


# Camera tilt motion control.
def cameraTiltCtrlCommand(commandType, valueInput):
    commandUp, commandDown = 0, 0
    if commandType == 'fb':
        if valueInput == -1:
            commandUp = 1
        elif valueInput == 1:
            commandDown = 1
    else:
        return

    if commandUp:
        ctrlArm.moveCde = 1
        ctrlArm.moveCin = 0
        ctrlArm.resume()
    elif commandDown:
        ctrlArm.moveCin = 1
        ctrlArm.moveCde = 0
        ctrlArm.resume()
    else:
        ctrlArm.moveCin = 0
        ctrlArm.moveCde = 0
        ctrlArm.resume()
        


# Robot movement stops.
def moveStop():
    global fb_input, lr_input
    fb_input = 0
    lr_input = 0
    robot.stop()


# Robot movement control.
def moveSmoothCtrl(commandType, valueInput):
    global fb_input, lr_input
    if commandType == 'y':
        fb_input = valueInput
    elif commandType == 'x':
        lr_input = valueInput
    else:
        return

    if abs(fb_input) < moveStartTor and abs(lr_input) < moveStartTor:
        moveStop()
        return
    leftSpeed  = -fb_input + lr_input
    rightSpeed = -fb_input - lr_input
    robot.left_motor.value  = leftSpeed
    robot.right_motor.value = rightSpeed

  
Multi-threaded control object for robotic arm.

In [6]:
# The multi-threaded control object for robotic arm, after instantiation, call pause() and resume() to control the pause and resume of run().
class ArmCtrlThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(ArmCtrlThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()
        self.moveXin = 0
        self.moveXde = 0
        self.moveYin = 0
        self.moveYde = 0
        self.moveCin = 0
        self.moveCde = 0

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            self.__flag.wait()
            armCtrl(self.moveXin, self.moveXde, self.moveYin, self.moveYde)
            if self.moveCin or self.moveCde:
                cameraTiltCtrl(self.moveCin, self.moveCde)
            if not self.moveXin and not self.moveXde and not self.moveYin and not self.moveYde and not self.moveCin and not self.moveCde:
                self.pause()
            sleep(movingTime)


# Instantiate and start the multi-threaded servo control thread.
ctrlArm = ArmCtrlThread()
ctrlArm.start()

  
### Gamepad basic control

In [7]:
import os, struct, array
from fcntl import ioctl
 
# Iterate over the joystick devices.
print('Available devices:')
for fn in os.listdir('/dev/input'):
    if fn.startswith('js'):
        print('  /dev/input/%s' % (fn))
 
 
# These constants were borrowed from linux/input.h
axis_names = {
    0x00 : 'x',
    0x01 : 'y',
    0x02 : 'z',
    0x03 : 'rx',
    0x04 : 'ry',
    0x05 : 'rz',
    0x06 : 'trottle',
    0x07 : 'rudder',
    0x08 : 'wheel',
    0x09 : 'gas',
    0x0a : 'brake',
    0x10 : 'lr',
    0x11 : 'fb',
    0x12 : 'hat1x',
    0x13 : 'hat1y',
    0x14 : 'hat2x',
    0x15 : 'hat2y',
    0x16 : 'hat3x',
    0x17 : 'hat3y',
    0x18 : 'pressure',
    0x19 : 'distance',
    0x1a : 'tilt_x',
    0x1b : 'tilt_y',
    0x1c : 'tool_width',
    0x20 : 'volume',
    0x28 : 'misc',
}
 
button_names = {
    0x120 : 'trigger',
    0x121 : 'thumb',
    0x122 : 'thumb2',
    0x123 : 'top',
    0x124 : 'top2',
    0x125 : 'pinkie',
    0x126 : 'base',
    0x127 : 'base2',
    0x128 : 'base3',
    0x129 : 'base4',
    0x12a : 'base5',
    0x12b : 'base6',
    0x12f : 'dead',
    0x130 : 'a',
    0x131 : 'b',
    0x132 : 'c',
    0x133 : 'x',
    0x134 : 'y',
    0x135 : 'z',
    0x136 : 'tl',
    0x137 : 'tr',
    0x138 : 'tl2',
    0x139 : 'tr2',
    0x13a : 'select',
    0x13b : 'start',
    0x13c : 'mode',
    0x13d : 'thumbl',
    0x13e : 'thumbr',
 
    0x220 : 'dpad_up',
    0x221 : 'dpad_down',
    0x222 : 'dpad_left',
    0x223 : 'dpad_right',
 
    0x2c0 : 'dpad_left',
    0x2c1 : 'dpad_right',
    0x2c2 : 'dpad_up',
    0x2c3 : 'dpad_down',
}
 
axis_map = []
button_map = []
 
# Open the joystick device.
fn = '/dev/input/js0'
print('Opening %s...' % fn)
jsdev = open(fn, 'rb')
 
# Get the device name.
buf = array.array('B', [0] * 64)
ioctl(jsdev, 0x80006a13 + (0x10000 * len(buf)), buf) # JSIOCGNAME(len)
js_name = buf.tobytes().rstrip(b'\x00').decode('utf-8')
print('Device name: %s' % js_name)
 
# Get number of axes and buttons.
buf = array.array('B', [0])
ioctl(jsdev, 0x80016a11, buf)
num_axes = buf[0]
 
buf = array.array('B', [0])
ioctl(jsdev, 0x80016a12, buf)
num_buttons = buf[0]
 
# Get the joystick address and correspond to the name in axis_names.
buf = array.array('B', [0] * 0x40)
ioctl(jsdev, 0x80406a32, buf) # JSIOCGAXMAP
 
for axis in buf[:num_axes]:
    axis_name = axis_names.get(axis, 'unknown(0x%02x)' % axis)
    axis_map.append(axis_name)

# Get the button address and correspond to the name in button_names.
buf = array.array('H', [0] * 200)
ioctl(jsdev, 0x80406a34, buf) # JSIOCGBTNMAP
 
for btn in buf[:num_buttons]:
    btn_name = button_names.get(btn, 'unknown(0x%03x)' % btn)
    button_map.append(btn_name)

print('%d axes found: %s' % (num_axes, ', '.join(axis_map)))
print('%d buttons found: %s' % (num_buttons, ', '.join(button_map)))
 

# Read the gamepad change information and control the robot.
class inputThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(inputThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            self.__flag.wait()
            evbuf = jsdev.read(8)
            if evbuf:
                time, value, type, number = struct.unpack('IhBB', evbuf)

                if type & 0x80:
                     print("(initial)", end="")

                if type & 0x01:
                    button = button_map[number]
                    if button:
                        grabCtrlCommand(button, value)
                        armCtrlCommand(button, value)
                        cameraPanCtrlCommand(button, value, 0)

                if type & 0x02:
                    axis = axis_map[number]
                    if axis:
                        moveSmoothCtrl(axis, value/32767.0)
                        cameraPanCtrlCommand(axis, int(round(value/32767.0,0)), 1)
                        cameraTiltCtrlCommand(axis, int(round(value/32767.0,0)))


# Instantiate the gamepad input thread and start reading information
inputThreading = inputThread()
inputThreading.start()
inputThreading.resume()

Available devices:
  /dev/input/js0
Opening /dev/input/js0...
Device name: ShanWan PC/PS3/Android
8 axes found: x, y, z, rz, gas, brake, lr, fb
16 buttons found: a, b, c, x, y, z, tl, tr, tl2, tr2, select, start, mode, thumbl, thumbr, unknown(0x13f)
(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)[130, -114]


# Key Description

L1 button: The gripper starts gripping.  
L2 button: The gripper starts loosening.  
A/B/X/Y button: The end point of the robotic arm retracts backward/the camera looks to the right/the camera looks to the left/the end point of the robotic arm extends forward.  
Left operation joystick: Control the front and back movement and steering of the robot.  
Left direction button (up): The camera looks up.  
Left direction button (down): The camera looks down.  
Left direction button (left): The camera looks left.  
Left direction button (right): The camera looks right.  
R1 button: The end point of the robotic arm moves upward in the vertical direction.  
R2 button: The end point of the robotic arm moves downward in the vertical direction.  

  
Again, let's close the camera conneciton properly so that we can use the camera in the later notebook.

In [7]:
camera.stop()